In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hypertension"
cohort = "GSE256539"

# Input paths
in_trait_dir = "../../input/GEO/Hypertension"
in_cohort_dir = "../../input/GEO/Hypertension/GSE256539"

# Output paths
out_data_file = "../../output/preprocess/Hypertension/GSE256539.csv"
out_gene_data_file = "../../output/preprocess/Hypertension/gene_data/GSE256539.csv"
out_clinical_data_file = "../../output/preprocess/Hypertension/clinical_data/GSE256539.csv"
json_path = "../../output/preprocess/Hypertension/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# Step 1: Gene Expression Data Availability
# Based on the background information, this is a digital spatial transcriptomics dataset
# It mentions "whole genome sequencing" and "genome-wide differential transcriptomic signature"
# So it's likely to contain gene expression data
is_gene_available = True

# Step 2: Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary, we don't see explicit trait (hypertension), age, or gender information
# The dataset seems to be comparing IPAH vs control samples, but individual characteristics are not provided
# Based on the background information, the dataset contains IPAH (Idiopathic Pulmonary Arterial Hypertension) patients vs. controls

# Let's look for trait (Hypertension) data
# Since the dataset is about IPAH, we could potentially infer IPAH vs control from the "individuial" field
# However, there's no clear indication which individuals are cases vs controls
trait_row = None  # No explicit trait information available for individuals

# Age data
age_row = None  # No age information available

# Gender data
gender_row = None  # No gender information available

# 2.2 Data Type Conversion
# Even though we don't have this information, we'll define placeholder conversion functions

def convert_trait(value: str) -> Optional[int]:
    """Convert trait value to binary format (0: control, 1: case)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Since we don't have explicit trait information, return None
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age value to continuous format"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Since we don't have age information, return None
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary format (0: female, 1: male)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Since we don't have gender information, return None
    return None

# Step 3: Save Metadata
# Initial filtering based on trait and gene availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical Feature Extraction
# Since trait_row is None, we skip this step
# If trait_row were not None, we would execute:
# clinical_df = geo_select_clinical_features(
#     clinical_df=clinical_data,
#     trait=trait,
#     trait_row=trait_row,
#     convert_trait=convert_trait,
#     age_row=age_row,
#     convert_age=convert_age,
#     gender_row=gender_row,
#     convert_gender=convert_gender
# )
# preview = preview_df(clinical_df)
# print("Clinical data preview:", preview)
# os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
# clinical_df.to_csv(out_clinical_data_file, index=False)


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# These identifiers appear to be human gene symbols.
# A2M is Alpha-2-Macroglobulin
# A4GALT is Alpha 1,4-Galactosyltransferase
# AAAS is Aladin WD Repeat Nucleoporin
# AACS is Acetoacetyl-CoA Synthetase
# etc.

# These are standard HGNC gene symbols for human genes, not probe IDs or other types of identifiers
# that would require mapping to gene symbols.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Since trait_row is None, as determined in Step 2, we cannot extract clinical features.
# Therefore, we cannot link clinical and genetic data.
# We'll proceed with the gene data only and properly report the absence of trait data.

# Check if trait data is available (from Step 2)
is_trait_available = trait_row is not None
print(f"Trait data availability: {is_trait_available}")

# 3. Since there is no trait data, we'll create a minimal dataframe with only gene data
# to satisfy the requirements of validate_and_save_cohort_info
minimal_df = pd.DataFrame(index=normalized_gene_data.columns[:5], 
                          data={gene: normalized_gene_data.loc[gene, :5] 
                                for gene in normalized_gene_data.index[:5]})

# We'll set is_biased to False since there's no trait to evaluate bias
is_biased = False

# 4. Conduct quality check and save the cohort information
note = "Dataset contains gene expression data from IPAH and control samples, but lacks individual trait, age, and gender annotations needed for associational studies."
validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=minimal_df,
    note=note
)

print("Dataset is not usable for trait-gene association studies due to missing trait information.")


### Step 6: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Since trait_row is None, as determined in Step 2, we cannot extract clinical features.
# Therefore, we cannot link clinical and genetic data.
# We'll proceed with the gene data only and properly report the absence of trait data.

# Check if trait data is available (from Step 2)
is_trait_available = trait_row is not None
print(f"Trait data availability: {is_trait_available}")

# 3. Create a minimal dataframe with only gene data to satisfy the requirements for validation
# We'll use a minimal representation of the gene data
minimal_df = pd.DataFrame(index=normalized_gene_data.columns[:5], 
                          columns=normalized_gene_data.index[:5])

# 4. Conduct quality check and save the cohort information
note = "Dataset contains gene expression data from IPAH and control samples, but lacks individual trait, age, and gender annotations needed for associational studies."
validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=is_trait_available, 
    is_biased=False,  # Set to False since there's no trait to evaluate bias
    df=minimal_df,
    note=note
)

print("Dataset is not usable for trait-gene association studies due to missing trait information.")